In [1]:
import collections
import re
import time
from tqdm import tqdm


import numpy as np

In [2]:
# считывание текстов Анны Карениной и Войны и Мира и объединение их

with open('corpora\AnnaKarenina.txt', encoding='utf-8') as f:
    anna_karenina = f.read()
    
with open('corpora\WarAndPeace.txt', encoding='utf-8') as f:
    war_and_peace = f.read()
    
text_full = anna_karenina + war_and_peace

## Задание 1
Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

In [20]:
# символы, с которыми работаем: буквы русского языка + пробел
SYMBOLS = "абвгдежзийклмнопрстуфхцчшщъыьэюя "


def normalize_text(text):
    """Нормализация текста: 
        * приведение все к строчным буквам 
        * оставляем только буквы русского алфавита и пробелы
    """
    
    text_norm = re.sub(r'[^а-я ]+', '', text.lower())
    
    return text_norm


def ngrams_calculation(text, n):
    """Расчет уни или биграм."""
    
    if n == 1:
        freq = collections.Counter(text)
    elif n == 2:   
        freq = collections.Counter(
            map(''.join, zip(text, text[1:]))
        )
    
    # отсортированные n-грамы по обуванию частоты встречаемости
    ngrams = list(
        dict(
            sorted(freq.items(), key=lambda item: item[1], reverse=True)
        ).keys()
    )
    
    s = sum(list(freq.values()))
    for key in freq:
        freq[key] /= s
    
    return freq, ngrams


def encode_text(text):
    """Кодирование текста."""
    
    symbols_permuted = np.random.choice(list(SYMBOLS), len(SYMBOLS), replace=False)

    encode_dict = {
        s1: s2 
        for s1, s2 in zip(SYMBOLS, symbols_permuted)
    }

    text_encoded = []
    for sym in text:
        text_encoded.append(encode_dict[sym])
            
    text_encoded = "".join(text_encoded)
    
    return text_encoded


def decode_text(text, ngrams_true):
    """Декодирование текста."""
    
    n = len(ngrams_true[0])
    
    freq, ngrams = ngrams_calculation(text, n)
    
    decode_dict = {
        s1: s2 
        for s1, s2 in zip(ngrams, ngrams_true)
    }

    text_decoded = []
    for i in range(0, len(text), n):
        if i == len(text) - 1 and n == 2:
            _, unigram_ = ngrams_calculation(text, n=1)
            text_decoded.append(
                unigram[np.argwhere(np.array(unigram_) == text[-1])[0][0]]
            )
        else:
            text_decoded.append(decode_dict[text[i:i+n]])

    text_decoded = "".join(text_decoded)
    
    return text_decoded
    
    
def accuracy(text1, text2):
    """Расчет точности."""
    
    acc = (np.array(list(text1)) == np.array(list(text2))).sum() / len(text1)
    
    return acc

#### Тестовый текст, который будем расшифровывать

In [21]:
text_test = """
Пьер сидел в гостиной, где Шиншин, как с приезжим из-за границы, 
завел с ним скучный для Пьера политический разговор, к которому 
присоединились и другие. Когда заиграла музыка, Наташа вошла 
в гостиную и, подойдя прямо к Пьеру, смеясь глазами и краснея, 
сказала:
- Мама велела вас просить танцевать.
- Я боюсь спутать фигуры, —сказал Пьер, —но ежели вы хотите быть моим учителем...
"""

**Работа с исходным текстом: нормализация, частоты букв**

In [22]:
# нормализация
text_full_norm = normalize_text(text_full)

# частоты символов
freq_unigram, unigram = ngrams_calculation(text_full_norm, n=1)

# топ-10 символов
unigram[:10]

[' ', 'о', 'е', 'а', 'н', 'и', 'т', 'с', 'л', 'в']

**Нормализация тестового текста**

In [23]:
text_test_norm = normalize_text(text_test)
text_test_norm

'пьер сидел в гостиной где шиншин как с приезжим изза границы завел с ним скучный для пьера политический разговор к которому присоединились и другие когда заиграла музыка наташа вошла в гостиную и подойдя прямо к пьеру смеясь глазами и краснея сказала мама велела вас просить танцевать я боюсь спутать фигуры сказал пьер но ежели вы хотите быть моим учителем'

**Кодирование**

In [24]:
text_test_encoded = encode_text(text_test_norm)
text_test_encoded

'ьцэтелуюэгехеяплиуапвеяюэебуабуаечычелеьтуэодуреуооыеятыаусжеоыхэгелеаурелчмфажвеюгзеьцэтыеьпгуиуфэлчуветыояпхптечечпиптпрмеьтулпэюуаугулцеуеютмяуэечпяюыеоыуятыгыерможчыеаыиыбыехпбгыехеяплиуамшеуеьпюпвюзеьтзрпечеьцэтмелрэзлцеягыоыруеуечтылаэзелчыоыгыерырыехэгэгыехылеьтплуицеиыасэхыицезеъпшлцельмиыицещуямтжелчыоыгеьцэтеапеэдэгуехжекпиуиэеъжицерпуремфуиэгэр'

**Декодирование**

In [25]:
text_test_decoded = decode_text(text_test_encoded, unigram)
text_test_decoded

'дмат иегас я унилевнз уга чевчев рор и дтеапйек еппо утовешы пояас и век ирьжвыз гсб дмато днсележаирез топунянт р рнлнтнкь дтеинагевесеим е гтьуеа рнуго поеутосо кьпыро волочо янчсо я унилевьх е днгнзгб дтбкн р дмать икабим усопоке е ртоиваб иропосо коко яасасо яои дтниелм ловшаяолм б юнхим идьлолм эеуьты иропос дмат вн айасе яы цнлела юылм кнек ьжеласак'

**Точность** (доля правильно расшифрованных букв)

In [26]:
accuracy(text_test_norm, text_test_decoded)

0.1792717086834734

## Задание 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм

In [27]:
# частоты биграм
freq_bigram, bigram = ngrams_calculation(text_full_norm, n=2)

# топ-10 биграм
bigram[:10]

['о ', 'и ', 'а ', 'е ', ' с', ' н', ' в', ' п', 'то', ' о']

In [47]:
text_test_decoded = [decode_text(text_test_encoded, unigram)[0], ]

for i in range(1, len(text_test_encoded)):
    
    letter = text_test_decoded[i - 1]
    
    for bgr in bigram:
        if bgr[0] == letter:
            text_test_decoded.append(bgr[1])
            break
            
# точность
accuracy(text_test_norm, text_test_decoded)

0.10364145658263306

> По биграмам точность стала хуже

## Задание 3
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.

**Используем итеративно следующий подход:**

* Меняем в закодированном тексте случайно выбранный символ на другой случайно выбранный символ, но уже из множества символов исходного большого текста. Если вдруг новый символ присутсвует уже в закодированном тексте, то просто поменяем их местами.

* Считаем логарифм вероятности полученного текста (последовательности символов) по биграмам - если логарифм вероятности увеличился, то принимаем замену. Иначе принимаем замену с вероятностью, пропорциональной разности вероятностей новой и предыдущей последовательностей.

* И так идем итеративно.

In [42]:
def log_proba(text, freq):
    """Логарифм вероятности данной последовательности символов."""
    
    eps = 1e-9
    logp = np.log(freq[text[:2]] + eps)
    
    for i in range(2, len(text)):
        
        s_prev = text[i - 1]
        
        p_sum = sum([freq[k] for k in freq if k[0] == s_prev])
        p = freq[text[i-1:i+1]]
        
        if p_sum == 0: 
            logp += np.log(eps)
        else:
            logp += np.log(p / p_sum + eps)
        
    return logp

In [35]:
text = text_test_encoded
logp = log_proba(text, freq_bigram)
s_temp = "#"

for j in range(10):
    
    acc = round(100 * accuracy(text, text_test_norm))
    print(f"iter={j}, log_prob={logp}, acc={acc}%")
    print("~" * 40)
    print(text)
    print("~" * 40)
    time.sleep(1)
    
            
    for i in tqdm(range(4000)):

        s_old, s_new = 0, 0

        while s_new == s_old:
            s_old = np.random.choice(list(text), 1)[0]
            s_new = np.random.choice(list(SYMBOLS), 1)[0]

        if s_new in text:
            text_new = text.replace(s_new, s_temp)
            text_new = text_new.replace(s_old, s_new)
            text_new = text_new.replace(s_temp, s_old)
        else:
            text_new = text.replace(s_old, s_new)

        logp_new = log_proba(text_new, freq_bigram)

        # print(logp, logp_new)

        if logp_new > logp or (np.exp(logp_new - logp) > np.random.rand()):
            logp = logp_new
            text = text_new

iter=0, log_prob=-2464.5184881632904, acc=0%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ьцэтелуюэгехеяплиуапвеяюэебуабуаечычелеьтуэодуреуооыеятыаусжеоыхэгелеаурелчмфажвеюгзеьцэтыеьпгуиуфэлчуветыояпхптечечпиптпрмеьтулпэюуаугулцеуеютмяуэечпяюыеоыуятыгыерможчыеаыиыбыехпбгыехеяплиуамшеуеьпюпвюзеьтзрпечеьцэтмелрэзлцеягыоыруеуечтылаэзелчыоыгыерырыехэгэгыехылеьтплуицеиыасэхыицезеъпшлцельмиыицещуямтжелчыоыгеьцэтеапеэдэгуехжекпиуиэеъжицерпуремфуиэгэр
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [05:51<00:00, 11.39it/s]


iter=1, log_prob=-995.9364172598758, acc=76%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
пыер синел д босвитой бне шитшит как с приезщим изза братиья задел с тим скучтяй нлю пыера поливический разбодор к коворому присоенитилисы и нрубие кобна заибрала музяка таваша дошла д босвитуэ и понойню прюмо к пыеру смеюсы блазами и крастею сказала мама делела дас просивы ватьедавы ю цоэсы спувавы гибуря сказал пыер то ещели дя ховиве цявы моим учивелем
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [05:51<00:00, 11.36it/s]


iter=2, log_prob=-914.4466310718153, acc=94%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
пьер сидел б гостиной где чинчин как с приезвим изза границы забел с ним скужный для пьера политижеский разгобор к которому присоединились и другие когда заиграла музыка натача бочла б гостиную и подойдя прямо к пьеру смеясь глазами и краснея сказала мама белела бас просить танцебать я фоюсь спутать шигуры сказал пьер но евели бы хотите фыть моим ужителем
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [03:15<00:00, 20.46it/s]


iter=8, log_prob=-899.2115817792014, acc=100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
пьер сидел в гостиной где шиншин как с приезжим изза границы завел с ним скучный для пьера политический разговор к которому присоединились и другие когда заиграла музыка наташа вошла в гостиную и подойдя прямо к пьеру смеясь глазами и краснея сказала мама велела вас просить танцевать я боюсь спутать щигуры сказал пьер но ежели вы хотите быть моим учителем
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [03:16<00:00, 20.32it/s]


iter=9, log_prob=-901.7800292100399, acc=97%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
пьер сидел в гостиной где чинчин как с приезшим изза границы завел с ним скужный для пьера политижеский разговор к которому присоединились и другие когда заиграла музыка натача вочла в гостиную и подойдя прямо к пьеру смеясь глазами и краснея сказала мама велела вас просить танцевать я боюсь спутать щигуры сказал пьер но ешели вы фотите быть моим ужителем
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [02:55<00:00, 22.76it/s]


> Получилось очень даже неплохо. Точность на 8-й итерации была 100% (с погрешностью до нескольких букв).

> Но результат сильно варьируется от запуска к запуску. Иногда заходит в некоторый локальный оптимум, где логарифм вероятности имеет близкое к полученному выше значению, но последовательность оказывается другой, не всегда хорошо читаемой.

## Задание 4

Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

> Здесь применим точно такой же подход, что и в задании 3.

In [39]:
text_secret = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [45]:
text = text_secret
logp = log_proba(text, freq_bigram)
s_temp = "#"

for j in range(5):
    
    print(f"iter={j}, log_prob={logp}")
    print("~" * 40)
    print(text)
    print("~" * 40)
    time.sleep(1)
    
            
    for i in tqdm(range(4000)):

        s_old, s_new = 0, 0

        while s_new == s_old:
            s_old = np.random.choice(list(text), 1)[0]
            s_new = np.random.choice(list(SYMBOLS), 1)[0]

        if s_new in text:
            text_new = text.replace(s_new, s_temp)
            text_new = text_new.replace(s_old, s_new)
            text_new = text_new.replace(s_temp, s_old)
        else:
            text_new = text.replace(s_old, s_new)

        logp_new = log_proba(text_new, freq_bigram)

        if logp_new > logp or (np.exp(logp_new - logp) > np.random.rand()):
            logp = logp_new
            text = text_new

iter=0, log_prob=-4745.627876660718
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [02:03<00:00, 32.42it/s]


iter=1, log_prob=-666.5960556385016
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
есба ни нашате лоряюбылий аба повта лоряюбылий текст у чтого соозмелац которий бегко проватюты скорее нсего ни нсе сшебюба прюнабыло а побувате яюксаяюбылий зюбб жю посбешлее ветнертое жюшюлае курсю дотц колевло ц лавего ле оземющ
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [01:55<00:00, 34.66it/s]


iter=2, log_prob=-569.6024942505843
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
если вы вимите норжальный или почти норжальный текст у ютого сообдения который легко прочитать скорее всего вы все смелали правильно и получите жаксижальный балл за послемнее четвертое замание курса шотя конечно я ничего не обедащ
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [02:05<00:00, 31.86it/s]


iter=3, log_prob=-563.1298220850088
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
если вы вимите нордальный или почти нордальный текст у ютого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса фотя конечно я ничего не обещаш
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [02:01<00:00, 32.82it/s]


iter=4, log_prob=-553.5928528568465
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаш
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [01:58<00:00, 33.63it/s]


In [46]:
text

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'